In [5]:
# Install wrapper for webscrape bot detection
pip install cloudscraper

Note: you may need to restart the kernel to use updated packages.


In [6]:
# Import package
import cloudscraper

# Check for connection
scraper = cloudscraper.create_scraper()
response = scraper.get('https://www.discogs.com/sell/list?format=Vinyl')

print(response.status_code)

200


In [8]:
# Import beautiful soup and SQL
from bs4 import BeautifulSoup
import sqlite3

In [20]:
# Create table in SQL to assign data
con = sqlite3.connect('dischogs-scrape')
cur = con.cursor()
cur.execute("create table dischogs_webscrape(raw_html text, parsed integer)")

In [23]:
# Validate SQL table
rows = cur.execute("select * from dischogs_webscrape")
rows.fetchone() is None

True

In [32]:
# Scrape website and save raw HTML to database
scraper = cloudscraper.create_scraper()
all_vinyl_data = []

for page_num in range(1, 8753):

    url = f'https://www.discogs.com/sell/list?sort=listed%2Cdesc&limit=250&format=Vinyl&ships_from=United+States&format_desc=LP&condition=Very+Good+Plus+%28VG%2B%29&page={page_num}'
    response = scraper.get(url)
    html_body = response.text
    cur.execute("insert into dischogs_webscrape (raw_html, parsed) values (?,?)", (html_body,0))
    con.commit()
    print(f"crawled page: {page_num}")

crawled page: 1
crawled page: 2
crawled page: 3
crawled page: 4
crawled page: 5
crawled page: 6
crawled page: 7
crawled page: 8
crawled page: 9
crawled page: 10
crawled page: 11
crawled page: 12
crawled page: 13
crawled page: 14
crawled page: 15
crawled page: 16
crawled page: 17
crawled page: 18
crawled page: 19
crawled page: 20
crawled page: 21
crawled page: 22
crawled page: 23
crawled page: 24
crawled page: 25
crawled page: 26
crawled page: 27
crawled page: 28
crawled page: 29
crawled page: 30
crawled page: 31
crawled page: 32
crawled page: 33
crawled page: 34
crawled page: 35
crawled page: 36
crawled page: 37
crawled page: 38
crawled page: 39
crawled page: 40
crawled page: 41
crawled page: 42
crawled page: 43
crawled page: 44
crawled page: 45
crawled page: 46
crawled page: 47
crawled page: 48
crawled page: 49
crawled page: 50
crawled page: 51
crawled page: 52
crawled page: 53
crawled page: 54
crawled page: 55
crawled page: 56
crawled page: 57
crawled page: 58
crawled page: 59
crawle

In [29]:
# Validate that our table contains data
rows = cur.execute("select * from dischogs_webscrape")
rows.fetchone() is None

False

In [30]:
rows.fetchone() 

('\n\n        \n\n<!DOCTYPE html>\n<html\n    class="is_not_mobile needs_reduced_ui "\n    lang="en"\n    xmlns:og="http://opengraphprotocol.org/schema/"\n    xmlns:fb="http://www.facebook.com/2008/fbml"\n>\n    <head>\n        <meta charset="utf-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n        <meta http-equiv="content-language" content="en">\n        <meta http-equiv="pragma" content="no-cache" />\n        <meta http-equiv="expires" content="-1" />\n\n\n                    <!-- OneTrust Cookies Consent Notice start for discogs.com and GPP Stub-->\n            <script src="https://cdn.cookielaw.org/scripttemplates/otSDKStub.js" data-language="en" type="text/javascript" charset="UTF-8" data-domain-script="05dc22ad-af2f-4fcc-865c-860f6ce05f91"></script>\n            <script type="text/javascript">\n                function OptanonWrapper() { }\n            </script>\n            <!-- OneTrust Cookies Consent Notice end for discogs.com -->\n                <!--

In [72]:
# Loop through HTML and compile data
all_vinyl_data = []

# Parse data from the SQLite table
rows = cur.execute("select * from dischogs_webscrape")
for row in rows:

    raw_html = row[0]
    soup = BeautifulSoup(raw_html)
    listings = soup.find_all('tr', class_='shortcut_navigable')
        
    for item in listings:
        name_tag = item.find('a', class_='item_description_title')
        price_tag = item.find('span', class_='price')
        rating_tag = item.find('div', class_='community_rating')
        have_tag = item.find('span', class_='community_label')
        want_tag = item.find_all('span', class_='community_label')[1:2]
        
                
        all_vinyl_data.append({
            'album': name_tag.text.strip() if name_tag else "N/A",
            'price': price_tag.text.strip() if price_tag else "N/A",
            'rating': rating_tag.text.strip() if rating_tag else "N/A",
            'have': have_tag.text.strip() if have_tag else "N/A",
            'want': want_tag[0].text.strip() if want_tag else "N/A"
        })


print(all_vinyl_data[200])

{'album': 'Karla Bonoff - Karla Bonoff (LP, Album)', 'price': '$5.99', 'rating': 'Rated: 3.74', 'have': '525 have', 'want': '26 want', 'genre': 'N/A'}


In [57]:
# Display scraped data
for vinyl in all_vinyl_data:
    print(vinyl)

{'album': 'Nilsson* - Knnillssonn (LP, Album)', 'price': '$15.00', 'rating': 'Rated: 4.02', 'have': '1340 have', 'want': '215 want'}
{'album': "Rockin' Horse (2) - Rockin' Horse (LP, Album)", 'price': '$10.00', 'rating': 'Rated: 4.26', 'have': '272 have', 'want': '212 want'}
{'album': 'The Beatles - Abbey Road (LP, Album, RE, RP, Rai)', 'price': '$30.00', 'rating': 'Rated: 4.64', 'have': '484 have', 'want': '1053 want'}
{'album': 'Rod Stewart - Foot Loose & Fancy Free (LP, Album, Jac)', 'price': '$3.99', 'rating': 'Rated: 3.70', 'have': '7628 have', 'want': '110 want'}
{'album': 'Bob Weir - Heaven Help The Fool (LP, Album, Ter)', 'price': '$6.00', 'rating': 'Rated: 3.90', 'have': '1770 have', 'want': '92 want'}
{'album': 'Keith Urban - Ripcord (LP, Album)', 'price': '$20.00', 'rating': 'Rated: 4.23', 'have': '816 have', 'want': '127 want'}
{'album': 'Aerosmith - Toys In The Attic (LP, Album, Pit)', 'price': '$8.00', 'rating': 'Rated: 4.02', 'have': '22835 have', 'want': '2603 want'}
{'

In [76]:
# Import pandas to convert data to csv
import pandas as pd

# Convert to dataframe and export to csv
vinyl_data = pd.DataFrame(all_vinyl_data)
vinyl_data.to_csv('discogs_vinyl_data.csv', index=False)
vinyl_data.

<bound method NDFrame.head of                                                    album   price       rating  \
0                     Nilsson* - Knnillssonn (LP, Album)  $15.00  Rated: 4.02   
1          Rockin' Horse (2) - Rockin' Horse (LP, Album)  $10.00  Rated: 4.26   
2      The Beatles - Abbey Road (LP, Album, RE, RP, Rai)  $30.00  Rated: 4.64   
3      Rod Stewart - Foot Loose & Fancy Free (LP, Alb...   $3.99  Rated: 3.70   
4       Bob Weir - Heaven Help The Fool (LP, Album, Ter)   $6.00  Rated: 3.90   
...                                                  ...     ...          ...   
10745  Grateful Dead* - Wake Of The Flood (LP, Album,...  $21.99  Rated: 4.80   
10746                  Les Dudek - Les Dudek (LP, Album)   $2.99  Rated: 4.00   
10747  Bob Seger & The Silver Bullet Band* - Live Bul...  $12.99  Rated: 3.93   
10748        Kenny Loggins - Nightwatch (LP, Album, Pit)   $6.99  Rated: 3.73   
10749  Balakirev*, Sir Thomas Beecham, Royal Philharm...  $13.34  Rated: 5.00  

In [90]:
import numpy as np

vd = vinyl_data[vinyl_data['want'].notna()]

vd.sort_values(by = 'want').tail()

,album,price,rating,have,want,genre
3498,The Sensational Nightingales - See You In The ...,$5.00,N/A,2 have,N/A,N/A
6575,"Sandi Patti* - Morning Like This (LP, Album)",$12.49,N/A,4 have,N/A,N/A
699,"Sophy - Más Sophy Que Nunca! (LP, Album)",$13.88,N/A,3 have,N/A,N/A
4518,The Claiborne Brothers Quartette - On Tour (LP...,$39.99,N/A,10 have,N/A,N/A
1649,"Paul ""Bear"" Bryant - The Legendary ""Bear"" (LP,...",$24.99,N/A,30 have,N/A,N/A
